# Lab2 - Sentiment Analysis with Scikit-Learn

The focus of this notebook is on performing sentiment analysis using the scikit-learn package. Material from [this notebook](http://www.pitt.edu/~naraehan/presentation/Movie+Reviews+sentiment+analysis+with+Scikit-Learn.html) was used.

**at the end of this notebook, you will be able to**:
* load the training data, i.e., the movie reviews
* inspect the training data, i.e., the movie reviews
* extracting features from the training data
* training and evaluating the *NaiveBayesClassifier*
* apply the classifier to fake movie reviews

**If you want to learn more, you might find the following link useful:**
* [documentation on dataset loading](http://scikit-learn.org/stable/datasets/)

In [1]:
import pathlib
import sklearn
import numpy
import nltk
from nltk.corpus import stopwords
from collections import Counter
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

## Loading the dataset
We are first going to load and inspect the **airlinetweets** dataset (which is part of the zip file you downloaded from Github).
We are going to use the method **load_files** as part of sklearn.
Let's first inspect what the help message of the function **load_files** states. Take your time to read the help instructions and to learn about the load_files function of sklearn.

In [2]:
# To get help use the next command!
help(load_files)

Help on function load_files in module sklearn.datasets.base:

load_files(container_path, description=None, categories=None, load_content=True, shuffle=True, encoding=None, decode_error='strict', random_state=0)
    Load text files with categories as subfolder names.
    
    Individual samples are assumed to be files stored a two levels folder
    structure such as the following:
    
        container_folder/
            category_1_folder/
                file_1.txt
                file_2.txt
                ...
                file_42.txt
            category_2_folder/
                file_43.txt
                file_44.txt
                ...
    
    The folder names are used as supervised signal label names. The individual
    file names are not important.
    
    This function does not try to extract features into a numpy array or scipy
    sparse matrix. In addition, if load_content is false it does not try to
    load the files in memory.
    
    To use text files in a scikit

Ok, so the function requires the following structure in order for it to work:
* container_folder/
    * category_1_folder/ (e.g., 'pos')
        * file_1.txt
        * file_2.txt
        * ...
        file_42.txt
    * category_2_folder/ (e.g., 'neg')
        * file_43.txt
        * file_44.txt
        * ...

Let's check whether our **airlinetweets** corpus has this structure.

In [3]:
cwd = pathlib.Path.cwd()
airline_tweets_folder = cwd.joinpath('airlinetweets')
print('path:', airline_tweets_folder)
print('this will print True if the folder exists:', 
      airline_tweets_folder.exists())

path: /Users/piek/Desktop/CBS2019/text-mining-ba/lab_sessions/lab2/airlinetweets
this will print True if the folder exists: True


In [4]:
str(airline_tweets_folder)

'/Users/piek/Desktop/CBS2019/text-mining-ba/lab_sessions/lab2/airlinetweets'

Inspect whether the corpus has the required structure.

....
How may categories are represented?
How would you add another one?

....

Hopefully, it is! Let's now load it using the function.

In [5]:
# loading all files as training data.
airline_tweets_train = load_files(str(airline_tweets_folder))

## Inspecting dataset

How many files do we have?

In [6]:
len(airline_tweets_train.data)

4755

In [7]:
# target names ("classes") are automatically generated from subfolder names
airline_tweets_train.target_names

['negative', 'neutral', 'positive']

How many do we have for each category?

In [8]:
freqs = Counter(airline_tweets_train.target)
for category, frequency in freqs.items():
    print(airline_tweets_train.target_names[category], frequency)

neutral 1515
positive 1490
negative 1750


In [9]:
# Let's inspect one file
airline_tweets_train.data[0]

b'@AmericanAir Why is your cover photo of TWA? Just wondering.'

In [10]:
# first file is in "neutral" folder
airline_tweets_train.filenames[0]

'/Users/piek/Desktop/CBS2019/text-mining-ba/lab_sessions/lab2/airlinetweets/neutral/AL_570069345818161152.txt'

In [11]:
# first file is a neutral review and is mapped to index 1 in target_names
airline_tweets_train.target[0]

1

We can find out what the index means by inserting it into **target_names**

In [12]:
airline_tweets_train.target_names[1]

'neutral'

## Extracting features from training data (see notebook Lab2-Feature representation.ipynb for more information)
Note: you might get a warning when you run the following cell. You do NOT have to resolve the warning.

In [13]:
# initialize airline object, and then turn airline tweets train data into a vector 

airline_vec = CountVectorizer(min_df=2, # If a token appears fewer times than this, across all documents, it will be ignored
                             tokenizer=nltk.word_tokenize, # we use the nltk tokenizer
                             stop_words=stopwords.words('english')) # stopwords are removed
airline_counts = airline_vec.fit_transform(airline_tweets_train.data)

In [14]:
# 'plane' is found in the corpus, mapped to index 1948
airline_vec.vocabulary_.get('plane')

1947

In [15]:
# large dimensions! 4,755 documents, 2902 unique terms. 
airline_counts.shape

(4755, 2901)

In [21]:
# print a single value in the matrix, most likely to be zero as it is a sparse matrix
print(airline_counts[5,5])

0


In [16]:
# Convert raw frequency counts into TF-IDF values
tfidf_transformer = TfidfTransformer()
airline_tfidf = tfidf_transformer.fit_transform(airline_counts)

In [17]:
# Same dimensions, now with tf-idf values instead of raw frequency counts
airline_tfidf.shape

(4755, 2901)

In [22]:
# the same cell as above now has a real value between 0.0 and 1.0, again most likely 0.0
print(airline_tfidf[5,5])

0.0


## Training and testing a Naive Bayes classifier
To train the classifier, we will first split the data into train and test.

In [23]:
# Now ready to build a classifier. 
# We will use Multinominal Naive Bayes as our model
from sklearn.naive_bayes import MultinomialNB

In [24]:
# Split data into training and test sets
# from sklearn.cross_validation import train_test_split  # deprecated in 0.18
from sklearn.model_selection import train_test_split

We choose 80% training and 20% test. 

In [25]:
docs_train, docs_test, y_train, y_test = train_test_split(
    airline_tfidf, # the tf-idf model
    airline_tweets_train.target, # the labels for each tweet 
    test_size = 0.20 # we use 80% for training and 20% for development
    ) 

One instance looks like this:

In [26]:
docs_train[55].toarray()

array([[0.        , 0.37404769, 0.        , ..., 0.        , 0.        ,
        0.        ]])

it's label is:

In [27]:
y_train[55]

2

which we know is then:

In [28]:
airline_tweets_train.target_names[y_train[55]]

'positive'

In [29]:
# Train a Multimoda Naive Bayes classifier
clf = MultinomialNB().fit(docs_train, y_train)

In [30]:
# Predicting the Test set results, find macro recall
y_pred = clf.predict(docs_test)

In [31]:
print('one tweet review:', airline_tweets_train.data[0])
print('gold label:', airline_tweets_train.target[0])
print('classifier predicted:', y_pred[0])

one tweet review: b'@AmericanAir Why is your cover photo of TWA? Just wondering.'
gold label: 1
classifier predicted: 2


See notebook **Lab2-Evaluation** for information on the evaluation. The micro recall of our classifier is:

In [32]:
sklearn.metrics.recall_score(y_true=y_test,
                             y_pred=y_pred,
                             average='micro')

0.8201892744479495

We can also inspect the least and most important features per category.

In [34]:
def important_features_per_class(vectorizer,classifier,n=80):
    class_labels = classifier.classes_
    feature_names =vectorizer.get_feature_names()
    topn_class1 = sorted(zip(classifier.feature_count_[0], feature_names),reverse=True)[:n]
    topn_class2 = sorted(zip(classifier.feature_count_[1], feature_names),reverse=True)[:n]
    topn_class3 = sorted(zip(classifier.feature_count_[2], feature_names),reverse=True)[:n]
    print("Important words in negative documents")
    for coef, feat in topn_class1:
        print(class_labels[0], coef, feat)
    print("-----------------------------------------")
    print("Important words in neutral documents")
    for coef, feat in topn_class2:
        print(class_labels[1], coef, feat) 
    print("-----------------------------------------")
    print("Important words in positive documents")
    for coef, feat in topn_class3:
        print(class_labels[2], coef, feat) 

# example of how to call from notebook:
important_features_per_class(airline_vec, clf)

Important words in negative documents
0 160.72275630128797 united
0 112.2073442767972 .
0 98.79234170705624 @
0 97.49655201448451 ``
0 56.325119535359896 flight
0 51.97956034583433 ?
0 45.91755252577125 #
0 43.836247099103375 !
0 39.44721235726242 n't
0 28.477065915318114 ''
0 23.88717386464075 service
0 22.857121763689662 virginamerica
0 21.76236558264745 's
0 21.588775192669267 cancelled
0 21.56463354329059 delayed
0 20.52035376418351 bag
0 20.44345278200883 ...
0 19.745566945017014 customer
0 19.099319471012162 get
0 18.55126125062186 hour
0 18.186927548171298 :
0 18.182707865007288 time
0 17.816540187489803 plane
0 17.6780903523429 hours
0 17.634155551985636 'm
0 17.162355412694428 late
0 16.197446183503807 gate
0 16.09205915224475 worst
0 15.897320223364014 still
0 15.86130918153623 waiting
0 15.615278187002664 airline
0 14.552867697466088 http
0 14.102561542991312 -
0 14.008315700369566 would
0 13.566014432122868 never
0 13.377359944506344 ;
0 13.361060564225628 delay
0 13.337940

## Applying classifier on our own data
Now we can apply our classifier to new data.
In the example below, these are movie reviews. In the exercise, you will choose tweets that you've selected.

In [35]:
# very short and fake movie reviews
reviews_new = ['This movie was excellent', 
               'Absolute joy ride', 
               'Steven Seagal was terrible', 
               'Steven Seagal shined through.', 
               'This was certainly a movie', 
               'Two thumbs up', 
               'I fell asleep halfway through', 
               "We can't wait for the sequel!!", 
               'I cannot recommend this highly enough', 
               'instant classic.', 
               'Steven Seagal was amazing.']
len(reviews_new)

11

In [36]:
# We re-use airline_vec to transform it in the same way as the training data
new_counts = airline_vec.transform(reviews_new)
new_counts.shape

(11, 2901)

In [37]:
# we compute tf idf values
reviews_new_tfidf = tfidf_transformer.transform(new_counts)

In [38]:
reviews_new_tfidf.shape

(11, 2901)

In [39]:
# have classifier make a prediction
pred = clf.predict(reviews_new_tfidf)

In [40]:
# print out results ()
for review, predicted_label in zip(reviews_new, pred):
    
    print('%s => %s' % (review, 
                        airline_tweets_train.target_names[predicted_label]))

This movie was excellent => positive
Absolute joy ride => positive
Steven Seagal was terrible => negative
Steven Seagal shined through. => negative
This was certainly a movie => negative
Two thumbs up => negative
I fell asleep halfway through => positive
We can't wait for the sequel!! => negative
I cannot recommend this highly enough => negative
instant classic. => negative
Steven Seagal was amazing. => positive
